In [44]:
import os
import glob
import pandas as pd
import numpy as np

In [45]:
import ast
csv_files = glob.glob('hashtags/*.{}'.format('csv'))

df_concat = pd.concat([pd.read_csv(f,usecols=(1,2)) for f in csv_files], ignore_index=True)

df_concat.to_csv('hashtags_merged.csv')

In [46]:
df = df_concat

# indexEmpty = df[ (df['hashtags'])].index

# df.drop(indexEmpty, inplace=True)
df = df[df.astype(str)['hashtags'] != '[]']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3330 entries, 0 to 3995
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   avatar    3330 non-null   object
 1   hashtags  3330 non-null   object
dtypes: object(2)
memory usage: 78.0+ KB


<class 'str'>
<class 'str'>


In [53]:
from wordsegment import load, segment
load()

def decomposed_hashtags(hashtags):
    return list(set(segment(hashtags)))
    # return set()

# # df_decomposed = df.head(100)
# #
# # df_decomposed['words'] = df_decomposed.apply(lambda row: decomposed_hashtags(row['hashtags']),axis=1)
# #
# # df_decomposed
# # df_100.to_csv('decomposed_hashtags_1000.csv')

In [54]:
import findspark
findspark.init()
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark                                 # only run after findspark.init()
from pyspark.sql import SparkSession
import urllib

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
spark_df = spark.createDataFrame(df.head(100))

In [55]:
spark_df.toPandas()

,avatar,hashtags
0,sinemunsal,"['#tbt', '#pink', '#Aydınlıkevler,', '#tb']"
1,ellekorea,"['#ELLECoverstar', '#제작비지원', '#공유', '#GONGYOO'..."
2,ashanegi,"['#mentalbehene', '#instareels', '#reelitfeeli..."
3,portadosfundos,"['#BBB21.', '#cozinhacompimenta', '#bbb', '#bb..."
4,tobibakre,"['#WellnessWithJumia', '#Camon16', '#CBeyond',..."
...,...,...
95,baranikster,"['#שירליוםראשון', '#טונה', '#🐟', '#לפיההנחיות'..."
96,movierapture,"['#Galaxyofentertainment', '#cinephile', '#mov..."
97,photo._.grammetry,"['#noodles', '#reelsinstagram', '#ketofy', '#r..."
98,hiddengemindia,"['#foodreels', '#cakeofinstagram', '#rasmalaic..."


In [56]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import IDF, Tokenizer, StopWordsRemover, CountVectorizer, StringIndexer

wordsData = spark_df

decompose_hashtags_udf = udf(lambda row: decomposed_hashtags(row), StringType())

filter_length_udf = udf(lambda row: [x for x in row if len(x) >= 3], ArrayType(StringType()))

wordsData = wordsData.withColumn('words',decompose_hashtags_udf(col('hashtags')))
wordsData =wordsData.withColumn('words_filtered',filter_length_udf(col('words')))

remover = StopWordsRemover(inputCol="words_filtered", outputCol = "words_without_stopwords", locale="en_US")
wordsData = remover.transform(wordsData)
wordsData.toPandas()

cv = CountVectorizer(inputCol="words_without_stopwords", outputCol="TF", vocabSize=1000) # keep the most 1000 common words
cvModel = cv.fit(wordsData)
wordsData = cvModel.transform(wordsData)

idf = IDF(inputCol="TF", outputCol="features")
idfModel = idf.fit(wordsData)
wordsData = idfModel.transform(wordsData)

wordsData.toPandas()

23/03/14 21:01:52 WARN StopWordsRemover: Default locale set was [en_RO]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


ERROR:root:KeyboardInterrupt while sending command.                 (1 + 7) / 8]
Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/opt/apache-spark/libexec/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/crangaantonio/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 